In [ ]:
import numpy as np
from numpy import r_

import cv2
from scipy.fftpack import dct,idct

q = np.array([[16,11,10,16,24,40,51,61],      
                    [12,12,14,19,26,58,60,55],    
                    [14,13,16,24,40,57,69,56],
                    [14,17,22,29,51,87,80,62],
                    [18,22,37,56,68,109,103,77],
                    [24,35,55,64,81,104,113,92],
                    [49,64,78,87,103,121,120,101],
                    [72,92,95,98,112,100,103,99]])

def to_bin(s):
    return ''.join(format(x,'08b') for x in bytearray(s,'utf-8'))
def unbin(s):
    data = bytes([int(s[i:i+8],2) for i in range(0,len(s),8)]).decode('utf-8')
    return data

def dct2(block):
    return dct(dct(block, axis = 0,norm ='ortho'),axis = 1, norm = 'ortho')

def idct2(block):
    res = idct(idct(block,axis = 0, norm ='ortho'),axis = 1, norm = 'ortho')
    return res

def wrong_dimensions(img):
    h = img.shape[0]
    w = img.shape[1]
    return h % 8 != 0 or w % 8 != 0

def resize(img):
    h = img.shape[0]
    w = img.shape[1] 
    img = cv2.resize(img,(w+(8 - w % 8),h+(8 - h % 8)))
    cv2.imwrite("cropped.bmp",img)
    return img

def quantize(block):
    return np.round(block/q)

def dequantize(block):
    return block*q

def compress(ch):
    ch_dim = ch.shape
    compressed_ch = np.zeros(ch_dim)
    for i in r_[:ch_dim[0]:8]:
        for j in r_[:ch_dim[1]:8]:
            compressed_ch[i:(i + 8), j:(j + 8)] = dct2(ch[i:(i + 8), j:(j + 8)])
            compressed_ch[i:(i + 8), j:(j + 8)] = quantize(compressed_ch[i:(i + 8), j:(j + 8)])
    return compressed_ch

def decompress(ch):
    ch_dim = ch.shape
    decompressed_ch = np.zeros(ch_dim)
    for i in r_[:ch_dim[0]:8]:
        for j in r_[:ch_dim[1]:8]:
            decompressed_ch[i:(i + 8), j:(j + 8)] = dequantize(ch[i:(i + 8), j:(j + 8)])
            decompressed_ch[i:(i + 8), j:(j + 8)] = np.round(idct2(decompressed_ch[i:(i + 8), j:(j + 8)]))
    return decompressed_ch

def lsb_hide(block,x):
    for i in range(8):
        if block[x][i] == 0 or block[x][i] == 1 :
            pass
        else:
            try:
                n = msg_bin.pop(0)
            except:
                break
            block[x][i] = int(block[x][i]) & ~1 | int(n,2)
    return block
         
def lsb_extract(block,x):
    res = []
    for i in range(8):
        if block[x][i] == 0 or block[x][i] == 1 :
            pass
        else:
            res.append(str(int(block[x][i]) & 1))
    return res
            
def hide(ch):
    ch_dim = ch.shape
    new_ch = np.zeros(ch_dim)
    for i in r_[:ch_dim[0]:8]:
        for j in r_[:ch_dim[1]:8]:
            new_ch[i:(i + 8), j:(j + 8)] = lsb_hide(ch[i:(i + 8), j:(j + 8)],3)
    return new_ch

def compress_c(img):
    b,g,r = cv2.split(img)
    b,g,r = compress(b),compress(g),compress(r)   
    b = hide(b)
    g = hide(g)
    r = hide(r)
    b,g,r = decompress(b),decompress(g),decompress(r)
    image = cv2.merge((b,g,r))
    return image

def extract(img):
    b,g,r = cv2.split(img)
    b,g,r = compress(b),compress(g),compress(r)
    res = []
    for ch in [b,g,r]:
        ch_dim = ch.shape
        new_ch = np.zeros(ch_dim)
        for i in r_[:ch_dim[0]:8]:
            for j in r_[:ch_dim[1]:8]:
                res+=lsb_extract(ch[i:(i + 8), j:(j + 8)],3)
    return res

#msg = "День подошел к концу. Благодаря вашим своевременным усилиям, люди, которые стремились помочь вам в ваших трудах, не успели подвергнуть себя опасности. В Театре Маски репетируют  новую пантомиму." 
msg = "- Are you William Blake?\n- Yes I am. Do you know my poetry?"
msg_bin = list(to_bin(msg))
len_msg = len(msg_bin)
#print(''.join(msg_bin))
    
img = cv2.imread("cropped.bmp",cv2.IMREAD_UNCHANGED)
if wrong_dimensions(img):
    img = resize(img)
img = compress_c(img)
cv2.imwrite("out.bmp",img)
img2 = cv2.imread("out.bmp",cv2.IMREAD_UNCHANGED)
res = extract(img2)
a = ''.join(res)[:len_msg]
print(unbin(a))

In [ ]:
messages = [
"bioluminescence","The tower, which was not","The tower, which was not supposed to be there, plunges",
"The tower, which was not supposed to be there, plunges into the earth in a place just before the black",
"The tower, which was not supposed to be there, plunges into the earth in a place just before the black pine\
forest begins to give way to swamp and then, ","The tower, which was not supposed to be there,plunges into the \
earth in a place just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats. Beyond the","The tower, which was not supposed to be there, plunges into\
the earth in a place just before the black pine forest begins to give way to swamp and then the reeds and \
wind-gnarled trees of the marsh flats. Beyond the marsh flats and the natural canals lies the ocean and, a  \
little farther down the coast, a derelict lighthouse.","bioluminescence "*100]
i = 0
for msg in messages:
    msg_bin = list(to_bin(msg))
    im = cv2.imread("cropped.bmp")
    res = compress_c(img)
    cv2.imwrite(str(i)+"out.bmp",res)
    i+=1

In [ ]:
#RMSE
def RMSE(a1, a2):
    return np.sqrt(((a1 - a2) ** 2).mean())
img1 = cv2.imread("cropped.bmp",cv2.IMREAD_UNCHANGED)
img2 = cv2.imread("out.bmp",cv2.IMREAD_UNCHANGED)
rmse = RMSE(img1,img2)
print("RMSE: {}".format(rmse))

rmse_values = []
i = 0
for msg in messages:
    im = cv2.imread("cropped.bmp")
    im2 = cv2.imread(str(i)+"out.bmp")
    rmse_values.append(RMSE(im,im2))
    i+=1
print(rmse_values)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.patches as patches

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': small,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
x = [len(msg.split(' ')) for msg in messages]
y = rmse_values
plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')
ax = plt.gca()
ax.set(xlabel='Message length (words)', ylabel='RMSE value')
plt.plot(x,y,c='tab:green')
plt.title("RMSE vs length", fontsize=22)
plt.xticks(x)
plt.show()
#plt.savefig("rmse_stats.pdf",bbox_inches='tight')